### **LLMs causales, instruction tuning, alineamiento y agentes**

Este cuaderno está **orientado a investigación aplicada** y cubre cuatro bloques integrados:

1. **Modelos de lenguaje causales**: tokenización, ventana de contexto y efectos experimentales.  
2. **Instruction tuning** (SFT): cómo ajustar un modelo base a formato instrucción-respuesta.  
3. **Alineamiento**: ideas de **RLHF**, **DPO/ORPO** y modelos de preferencia.  
4. **LLMs como agentes**: herramientas, memoria, planificación y evaluación del comportamiento.

> **Nota:** algunas celdas son **demostraciones ligeras** (corren localmente), y otras son **plantillas** para ejecutar con GPU/Colab/servidores de laboratorio.


### **0. Configuración y reproducibilidad**


#### **0.1 Principios mínimos**
En investigación con LLMs, documenta siempre:

- **modelo base** (nombre y versión),
- **tokenizer** (mismo repo del modelo),
- **dataset** (origen + fecha + versión),
- **prompt template**,
- **semilla** (`seed`),
- **hardware** (GPU/VRAM),
- **métricas** y scripts de evaluación,
- **criterio de selección** (mejor checkpoint por qué métrica).

Esto evita resultados "bonitos pero irreproducibles".


In [ ]:
# Reproducibilidad básica
import os
import random
import math
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Tuple, Optional

import numpy as np

try:
    import torch
except Exception:
    torch = None

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

if torch is not None:
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)

print(f"Seed fija: {SEED}")
print(f"Torch disponible: {torch is not None}")
if torch is not None:
    print(f"CUDA disponible: {torch.cuda.is_available()}")


In [ ]:
# Registro sencillo de experimentos (sin depender de Weights & Biases)
import json
from pathlib import Path
from datetime import datetime

LOG_DIR = Path("logs_investigacion_llm")
LOG_DIR.mkdir(exist_ok=True)

def registrar_experimento(nombre: str, config: Dict[str, Any], resultados: Dict[str, Any]) -> str:
    timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    path = LOG_DIR / f"{timestamp}_{nombre}.json"
    payload = {
        "timestamp_utc": timestamp,
        "nombre": nombre,
        "config": config,
        "resultados": resultados,
    }
    path.write_text(json.dumps(payload, indent=2, ensure_ascii=False), encoding="utf-8")
    return str(path)

# Ejemplo rápido
ruta_log = registrar_experimento(
    "demo_setup",
    {"seed": SEED, "objetivo": "verificar formato de logs"},
    {"status": "ok"}
)
print("Log guardado en:", ruta_log)


### **1. Modelos de lenguaje causales, tokenización y ventana de contexto**


#### **1.1 Modelo causal (decoder-only)**
Un **LLM causal** predice el siguiente token:

$$
p(x_1, x_2, ..., x_T)=\prod_{t=1}^{T} p(x_t \mid x_{<t})
$$

Esto lo hace ideal para:
- generación de texto,
- chat,
- código,
- planificación textual (agents),
- tool-use via prompts o llamadas estructuradas.

##### **Pregunta de investigación**

¿Cómo cambia el rendimiento cuando:
- varía la **tokenización**,
- aumenta/disminuye la **ventana de contexto**,
- cambia el formato del prompt?


#### **1.2 Tokenización (demostración ligera)**
Primero usamos una tokenización simple para visualizar conceptos. Luego mostramos cómo hacerlo con **Hugging Face** (si tienes acceso al modelo/tokenizer).


In [ ]:
# Tokenización muy simple
import re
from collections import Counter

def tokenize_simple(texto: str) -> List[str]:
    # Separa palabras y signos de puntuación de forma básica
    return re.findall(r"\w+|[^\w\s]", texto, re.UNICODE)

texto = "Los modelos causales predicen el siguiente token. ¡Eso afecta la ventana de contexto!"
tokens = tokenize_simple(texto)

print("Texto:", texto)
print("Tokens:", tokens)
print("N° tokens:", len(tokens))


In [ ]:
# Comparación rápida entre textos de distinta granularidad
textos = [
    "Hola mundo.",
    "Los LLMs pueden razonar mejor con prompts estructurados.",
    "def suma(a, b): return a + b"
]

for t in textos:
    toks = tokenize_simple(t)
    print("-" * 70)
    print("Texto:", t)
    print("Tokens:", toks)
    print("Longitud:", len(toks))


#### **1.3 Tokenización con Hugging Face (opcional)**
Esta celda usa `transformers` y un tokenizer real. Si no estás en Colab o no tienes internet, puedes saltarla.


In [ ]:
# Tokenización real con Hugging Face (opcional)
try:
    from transformers import AutoTokenizer
    tokenizer_name = "gpt2"  # cambia por un tokenizer de tu modelo base
    hf_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    muestras = [
        "La tokenización subword afecta longitud, costo y truncamiento.",
        "Instruction tuning requiere un template consistente.",
        "¿Qué pasa con DPO/ORPO cuando el prompt es ambiguo?"
    ]

    for s in muestras:
        ids = hf_tokenizer.encode(s, add_special_tokens=False)
        toks = hf_tokenizer.convert_ids_to_tokens(ids)
        print("=" * 80)
        print("Texto:", s)
        print("IDs:", ids)
        print("Tokens:", toks)
        print("N° tokens:", len(ids))
except Exception as e:
    print("No se pudo ejecutar la tokenización Hugging Face (opcional).")
    print("Detalle:", e)


#### **1.4 Ventana de contexto: truncamiento vs sliding window**
En investigación, una fuente común de errores es **truncar silenciosamente**.  
Hay que medir:

- cuántos ejemplos fueron truncados,
- cuánto contenido se perdió,
- si el truncamiento sesga ciertas clases/tipos de prompts.


In [ ]:
# Funciones de ventana de contexto
from typing import Iterable

def truncar_tokens(tokens: List[str], max_len: int) -> List[str]:
    return tokens[:max_len]

def sliding_windows(tokens: List[str], window: int, stride: int) -> List[List[str]]:
    ventanas = []
    i = 0
    while i < len(tokens):
        chunk = tokens[i:i+window]
        if not chunk:
            break
        ventanas.append(chunk)
        if i + window >= len(tokens):
            break
        i += stride
    return ventanas

texto_largo = " ".join(["contexto"] * 45) + " fin"
tokens_largos = tokenize_simple(texto_largo)

print("Total tokens:", len(tokens_largos))
print("Truncado a 16:", truncar_tokens(tokens_largos, 16))

ventanas = sliding_windows(tokens_largos, window=16, stride=8)
print("\nN° ventanas:", len(ventanas))
for i, v in enumerate(ventanas[:3]):
    print(f"Ventana {i}: len={len(v)}")


In [ ]:
# Métricas de truncamiento sobre un mini-corpus
corpus = [
    " ".join(["a"] * 8),
    " ".join(["b"] * 20),
    " ".join(["c"] * 32),
    " ".join(["d"] * 70),
]

MAX_LEN = 16
stats = []
for i, txt in enumerate(corpus):
    toks = tokenize_simple(txt)
    n = len(toks)
    truncados = max(0, n - MAX_LEN)
    stats.append({
        "ejemplo": i,
        "tokens": n,
        "truncados": truncados,
        "fraccion_truncada": truncados / n if n else 0.0,
    })

for row in stats:
    print(row)

prom_trunc = sum(r["fraccion_truncada"] for r in stats) / len(stats)
print("\nFracción truncada promedio:", round(prom_trunc, 4))


#### **1.5 Demostración de un pequeño modelo causal**
Esta demostración no reemplaza a un LLM real, pero ayuda a entender la lógica causal:
- entrada: secuencia
- objetivo: **siguiente token**
- pérdida: `CrossEntropy` desplazando en 1 posición


In [ ]:
# Modelo causal a nivel de caracteres
# Nota: se entrena sobre texto pequeño, solo para entender el flujo.
if torch is None:
    print("Torch no disponible; salta esta celda.")
else:
    import torch.nn as nn
    import torch.nn.functional as F

    texto_toy = "los llm causales predicen el siguiente token. " * 30
    vocab = sorted(list(set(texto_toy)))
    stoi = {ch:i for i,ch in enumerate(vocab)}
    itos = {i:ch for ch,i in stoi.items()}
    data = torch.tensor([stoi[ch] for ch in texto_toy], dtype=torch.long)

    block_size = 24
    def batch_toy(batch_size=16):
        ix = torch.randint(0, len(data)-block_size-1, (batch_size,))
        x = torch.stack([data[i:i+block_size] for i in ix])
        y = torch.stack([data[i+1:i+block_size+1] for i in ix])
        return x, y

    class MiniCausalLM(nn.Module):
        def __init__(self, vocab_size, d_model=64):
            super().__init__()
            self.emb = nn.Embedding(vocab_size, d_model)
            self.pos = nn.Embedding(block_size, d_model)
            self.ln = nn.LayerNorm(d_model)
            self.head = nn.Linear(d_model, vocab_size)

        def forward(self, x, y=None):
            B, T = x.shape
            tok = self.emb(x)
            pos = self.pos(torch.arange(T, device=x.device))[None, :, :]
            h = self.ln(tok + pos)
            logits = self.head(h)  # [B, T, V]
            loss = None
            if y is not None:
                loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1))
            return logits, loss

    model = MiniCausalLM(vocab_size=len(vocab))
    opt = torch.optim.AdamW(model.parameters(), lr=1e-2)

    model.train()
    for step in range(120):
        xb, yb = batch_toy()
        logits, loss = model(xb, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if step % 30 == 0:
            print(f"step={step:03d} loss={loss.item():.4f}")

    # Generación autoregresiva simple
    model.eval()
    contexto = torch.tensor([[stoi[c] for c in "los llm "]], dtype=torch.long)
    out = contexto.clone()

    for _ in range(60):
        x = out[:, -block_size:]
        logits, _ = model(x)
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        out = torch.cat([out, next_id], dim=1)

    generado = "".join(itos[i] for i in out[0].tolist())
    print("\nTexto generado:")
    print(generado)


#### **1.6 Métricas relevantes para este bloque**
Para reportes de investigación:

- **Longitud en tokens** (prompt, respuesta, total)
- **% truncado**
- **Costo por ejemplo** (tokens procesados)
- **Perplejidad / NLL** (si aplica)
- **Latencia de generación**
- **Tasa de errores de formato** (si pides JSON/tool-call)

> En muchos proyectos, mejorar tokenización/prompt/contexto da más ganancia que "entrenar más".


### **2. Instrucción y ajuste fino sobre modelos preentrenados (Instruction Tuning/SFT)**


#### **2.1 Qué es instruction tuning**

Partimos de un modelo base preentrenado (causal) y lo ajustamos con ejemplos tipo:

- **instrucción** (qué se pide),
- **entrada** opcional (contexto),
- **salida** (respuesta deseada).

Esto se suele llamar:
- **SFT** (Supervised Fine-Tuning),
- **instruction tuning**,
- **instruct tuning**.

##### **Hipótesis de investigación**
Un mejor *template de prompt* y una mejor *curación de datos* pueden mejorar más que aumentar parámetros entrenables.


#### **2.2 Formato de datos (estándar mínimo)**
Usaremos un formato simple compatible con datasets JSONL:


In [ ]:
# Dataset sintético de ejemplo (para estructura, no para calidad real)
dataset_sft = [
    {
        "instruction": "Resume el siguiente texto en una oración.",
        "input": "Los transformadores usan atención para modelar dependencias de largo alcance.",
        "output": "Los transformadores usan atención para capturar dependencias largas en secuencias."
    },
    {
        "instruction": "Clasifica el sentimiento (positivo/negativo).",
        "input": "El curso estuvo muy bien organizado y aprendí bastante.",
        "output": "positivo"
    },
    {
        "instruction": "Extrae una lista JSON de tecnologías mencionadas.",
        "input": "Usamos PyTorch, Hugging Face, FastAPI y Docker en el proyecto.",
        "output": "[\"PyTorch\", \"Hugging Face\", \"FastAPI\", \"Docker\"]"
    },
]

dataset_sft


In [ ]:
# Template de prompt para modelo causal
def formatear_prompt_sft(ej: Dict[str, str]) -> str:
    inst = ej["instruction"].strip()
    inp = ej.get("input", "").strip()
    out = ej["output"].strip()

    if inp:
        return (
            "### Instrucción:\n"
            f"{inst}\n\n"
            "### Entrada:\n"
            f"{inp}\n\n"
            "### Respuesta:\n"
            f"{out}"
        )
    else:
        return (
            "### Instrucción:\n"
            f"{inst}\n\n"
            "### Respuesta:\n"
            f"{out}"
        )

for ej in dataset_sft:
    print("=" * 80)
    print(formatear_prompt_sft(ej))


#### **2.3 Preparación de etiquetas (masking)**
En SFT con modelos causales, normalmente queremos que la pérdida se calcule **solo sobre la respuesta del asistente**, no sobre toda la instrucción.

Eso se logra con **label masking**:
- tokens del prompt -> `-100`
- tokens de respuesta -> ids reales


In [ ]:
# Demostración conceptual de label masking sin depender de HF
def construir_labels_masked(tokens_prompt: List[int], tokens_respuesta: List[int]) -> Tuple[List[int], List[int]]:
    input_ids = tokens_prompt + tokens_respuesta
    labels = [-100] * len(tokens_prompt) + tokens_respuesta[:]  # solo aprende la respuesta
    return input_ids, labels

# Ejemplo con IDs ficticios
prompt_ids = [10, 11, 12, 13]
resp_ids = [50, 51, 52]
input_ids, labels = construir_labels_masked(prompt_ids, resp_ids)

print("input_ids:", input_ids)
print("labels   :", labels)


#### **2.4 Plantilla de SFT con Hugging Face + TRL (PEFT/LoRA)**
Esta sección es una **plantilla de investigación**.  
Adáptala con tu modelo base y tu dataset real.


In [ ]:
# Plantilla SFT con Transformers/TRL/PEFT (opcional, requiere dependencias y GPU)
# Sugerido para Colab/A100/Lab con CUDA.

# !pip install -U transformers datasets trl peft accelerate bitsandbytes

sft_template = r'''
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig

model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # ejemplo (ajústalo)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # device_map="auto", torch_dtype="auto"  # opcional según hardware
)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # depende del modelo
)

# Si tu dataset está en JSONL con columnas instruction/input/output:
# ds = load_dataset("json", data_files={"train": "train.jsonl", "validation": "val.jsonl"})

def format_example(example):
    instruction = example["instruction"].strip()
    input_text = (example.get("input") or "").strip()
    output_text = example["output"].strip()
    if input_text:
        return f"""### Instrucción:
{instruction}

### Entrada:
{input_text}

### Respuesta:
{output_text}"""
    return f"""### Instrucción:
{instruction}

### Respuesta:
{output_text}"""

# ds = ds.map(lambda x: {"text": format_example(x)})

cfg = SFTConfig(
    output_dir="outputs_sft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    max_seq_length=1024,
    packing=False,
    report_to=[],  # agrega "wandb" si lo usas
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds.get("validation"),
    args=cfg,
    peft_config=peft_config,
    dataset_text_field="text",
)

trainer.train()
trainer.save_model("outputs_sft/final")
'''
print(sft_template[:2000])
print("\n... (plantilla completa en la variable sft_template)")


#### **2.5 QLoRA (orientación de investigación)**
Si el modelo es más grande y tu VRAM es limitada, usa **QLoRA**:

- pesos base cuantizados (4-bit),
- adaptadores LoRA entrenables,
- costo menor de VRAM.

##### Variables de investigación útiles
- `r` (rank LoRA)
- `target_modules`
- `max_seq_length`
- `packing`
- mezcla de datasets
- proporción de ejemplos largos/cortos


In [ ]:
# Plantilla conceptual de configuración QLoRA (no ejecuta por sí sola)
qlora_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": True,
    "bnb_4bit_compute_dtype": "bfloat16",  # o float16 según hardware
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
}
qlora_config


#### **2.6 Evaluación de SFT (mínimo experimental)**
No evalúes solo con "se ve bien". Usa al menos:

- **Exact match / F1** (si la tarea lo permite),
- **ROUGE / BLEU** (si es resumen o traducción, con cuidado),
- **format compliance** (JSON válido, campos correctos),
- **human eval pequeña** (rubrica),
- **errores frecuentes** (halucinación, longitud, formato).


In [ ]:
# Evaluación de formato JSOn
import json

predicciones = [
    '{"tecnologias": ["PyTorch", "Docker"]}',
    '{"tecnologias": ["FastAPI", "Hugging Face"]}',
    'No es JSON'
]

def es_json_valido(s: str) -> bool:
    try:
        json.loads(s)
        return True
    except Exception:
        return False

validas = [es_json_valido(p) for p in predicciones]
print("Predicciones válidas:", validas)
print("Tasa de formato válido:", sum(validas) / len(validas))


### **3. Alineamiento: RLHF, DPO/ORPO y modelos de preferencia**


#### **3.1 Panorama conceptual**
#### **RLHF** (visión clásica)

Pipeline típico:

1. **SFT** (modelo instruccional base)  
2. Recolectar **preferencias humanas** (A mejor que B)  
3. Entrenar un **modelo de preferencia/recompensa**  
4. Optimizar la política (PPO u otra técnica)

#### **DPO/ORPO**

Evitan parte de la complejidad de RLHF clásico:
- **DPO** optimiza directamente con pares preferidos/rechazados.
- **ORPO** integra una señal de preferencia junto con el objetivo supervisado.

> En práctica, DPO/ORPO suelen ser más simples de reproducir que PPO-RLHF.


#### **3.2 Esquema de datos de preferencia**

Formato típico de ejemplo:

- `prompt`
- `chosen` (respuesta preferida)
- `rejected` (respuesta menos preferida)

La calidad del dataset de preferencias es crítica.


In [ ]:
# Dataset de preferencias
dataset_pref = [
    {
        "prompt": "Explica qué es una ventana de contexto en 2 líneas.",
        "chosen": "La ventana de contexto es la cantidad de tokens que el modelo puede considerar a la vez. Si el texto excede ese límite, parte del contenido se trunca o se procesa por fragmentos.",
        "rejected": "Es la memoria del modelo para siempre y nunca se pierde información."
    },
    {
        "prompt": "Devuelve una lista JSON con 3 frutas.",
        "chosen": '["manzana", "pera", "uva"]',
        "rejected": "manzana, pera, uva"
    },
]
dataset_pref


#### **3.3 Modelo de preferencia (reward/preference model)**

En RLHF clásico, el reward model (modelo de recompensa) aprende a asignar un score a respuestas.  Aquí hacemos una demostración ligera con rasgos simples para entender el flujo (no para uso real).


In [ ]:
# Demostración ligera: "modelo de preferencia" heurístico (no ML real)
# Sirve para entender cómo se generan señales de preferencia.
import math

def score_respuesta_heuristico(prompt: str, respuesta: str) -> float:
    score = 0.0
    # Penaliza respuestas demasiado cortas
    n = len(respuesta.split())
    score += min(n / 15.0, 1.5)
    # Bonifica formato JSON si el prompt lo pide
    if "json" in prompt.lower():
        try:
            json.loads(respuesta)
            score += 1.0
        except Exception:
            score -= 0.5
    # Penaliza afirmaciones absolutas dudosas (ejemplo didáctico)
    if "nunca" in respuesta.lower() and "siempre" in respuesta.lower():
        score -= 0.3
    return score

for ej in dataset_pref:
    s_ch = score_respuesta_heuristico(ej["prompt"], ej["chosen"])
    s_rj = score_respuesta_heuristico(ej["prompt"], ej["rejected"])
    print("=" * 80)
    print("Prompt:", ej["prompt"])
    print("Score chosen  :", round(s_ch, 3))
    print("Score rejected:", round(s_rj, 3))
    print("¿Chosen mejor?:", s_ch > s_rj)


#### **3.4 Pérdida DPO (demostración matemática con log-probabilidades)**
La idea central de DPO: aumentar la preferencia del modelo por `chosen` frente a `rejected`, comparado contra un modelo de referencia.

Para entenderlo, simulamos log-probabilidades.


In [ ]:
# Demostración de pérdida DPO con tensores (sin modelo real)
if torch is None:
    print("Torch no disponible; salta esta celda.")
else:
    import torch.nn.functional as F

    # Log-probs simuladas por ejemplo (sumadas sobre la respuesta)
    logp_pi_chosen = torch.tensor([ -5.2, -3.1, -7.8 ])
    logp_pi_rej    = torch.tensor([ -6.1, -2.6, -8.7 ])
    logp_ref_chosen= torch.tensor([ -5.5, -3.0, -7.9 ])
    logp_ref_rej   = torch.tensor([ -5.9, -2.8, -8.5 ])

    beta = 0.1

    # Δπ y Δref
    delta_pi = logp_pi_chosen - logp_pi_rej
    delta_ref = logp_ref_chosen - logp_ref_rej

    # Pérdida DPO: -log(sigmoid(beta * (Δπ - Δref)))
    loss = -F.logsigmoid(beta * (delta_pi - delta_ref)).mean()

    print("delta_pi :", delta_pi)
    print("delta_ref:", delta_ref)
    print("DPO loss :", float(loss))


#### **3.5 ORPO (idea + demo conceptual)**

ORPO combina:
- objetivo supervisado (NLL de la respuesta correcta),
- una señal de preferencia (odds ratio/ranking).

No implementaremos la fórmula completa aquí, pero sí una versión conceptual para ver el balance de términos.


In [ ]:
# Demostración conceptual de combinación SFT + preferencia (estilo ORPO simplificado)
if torch is None:
    print("Torch no disponible; salta esta celda.")
else:
    # Simulamos dos términos:
    nll_sft = torch.tensor(1.85)          # pérdida de lenguaje
    pref_penalty = torch.tensor(0.42)     # término de preferencia (ejemplo)
    alpha = 0.5                           # peso de la preferencia

    loss_total = nll_sft + alpha * pref_penalty

    print("NLL SFT         :", float(nll_sft))
    print("Término prefer. :", float(pref_penalty))
    print("alpha           :", alpha)
    print("Loss total      :", float(loss_total))


#### **3.6 Plantillas DPO/ORPO con TRL (investigación)**
Estas son plantillas de alto nivel para adaptar a tu entorno.


In [ ]:
# Plantilla DPO con TRL (opcional)
dpo_template = r'''
# !pip install -U transformers datasets trl peft accelerate bitsandbytes

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig

model_name = "Qwen/Qwen2.5-1.5B-Instruct"  # ejemplo
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(model_name)

# Dataset con columnas: prompt, chosen, rejected
# ds = load_dataset("json", data_files={"train": "pref_train.jsonl", "validation": "pref_val.jsonl"})

cfg = DPOConfig(
    output_dir="outputs_dpo",
    beta=0.1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=5e-6,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    max_length=1024,
    max_prompt_length=512,
    report_to=[],
)

trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=cfg,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds.get("validation"),
)

trainer.train()
trainer.save_model("outputs_dpo/final")
'''
print(dpo_template[:1800])
print("\n... (plantilla completa en la variable dpo_template)")


In [ ]:
# Plantilla ORPO con TRL (opcional, si tu versión de TRL lo soporta)
orpo_template = r'''
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import ORPOTrainer, ORPOConfig

model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

cfg = ORPOConfig(
    output_dir="outputs_orpo",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    learning_rate=5e-6,
    num_train_epochs=1,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=100,
    max_length=1024,
    max_prompt_length=512,
    report_to=[],
    # beta / otros parámetros dependen de la versión
)

trainer = ORPOTrainer(
    model=model,
    args=cfg,
    tokenizer=tokenizer,
    train_dataset=ds["train"],      # columnas: prompt, chosen, rejected
    eval_dataset=ds.get("validation"),
)

trainer.train()
trainer.save_model("outputs_orpo/final")
'''
print(orpo_template[:1800])
print("\n... (plantilla completa en la variable orpo_template)")


#### **3.7 Qué medir en alineamiento**

Además de métricas de tarea, mide:

- **Win rate** en pares de preferencia (vs baseline)
- **Format compliance** (muy importante)
- **Toxicidad / seguridad** (si aplica)
- **Tasa de rechazo apropiado** (cuando la petición es riesgosa o inválida)
- **Over-optimization** (respuestas demasiado largas/complacientes)
- **Robustez a prompts ambiguos**

##### **Riesgo común**
Optimizar solo por "preferencia" puede degradar **exactitud factual**.


### **4. LLM como agentes: herramientas, memoria y planificación**


#### **4.1 Qué entendemos por "agente"**
En este cuaderno, un agente es un sistema que combina:

- un **modelo de lenguaje** (razonamiento/generación),
- **herramientas** (calculadora, búsqueda, base de datos, código),
- **memoria** (estado de conversación, hechos, resultados previos),
- **planificación** (dividir tareas, decidir siguiente acción).

> Investigación real aquí = medir si el uso de herramientas mejora la tarea, no solo demostrar "que llama una función".


#### **4.2 Herramientas**
Primero creamos herramientas locales y observables (con trazas).


In [ ]:
# Herramientas con trazas
from dataclasses import dataclass, field

@dataclass
class ToolCall:
    nombre: str
    entrada: Dict[str, Any]
    salida: Any

@dataclass
class TrazaAgente:
    pasos: List[str] = field(default_factory=list)
    tool_calls: List[ToolCall] = field(default_factory=list)

    def log(self, msg: str):
        self.pasos.append(msg)

    def registrar_tool(self, nombre: str, entrada: Dict[str, Any], salida: Any):
        self.tool_calls.append(ToolCall(nombre, entrada, salida))

def tool_calculadora(expr: str) -> Dict[str, Any]:
    try:
        # Demo controlada: eval local básico
        resultado = eval(expr, {"__builtins__": {}}, {})
        return {"ok": True, "resultado": resultado}
    except Exception as e:
        return {"ok": False, "error": str(e)}

BASE_KB = {
    "rlhf": "RLHF suele incluir SFT, modelo de preferencia y optimización de política.",
    "dpo": "DPO optimiza preferencias con pares chosen/rejected sin PPO clásico.",
    "orpo": "ORPO integra objetivo supervisado y señal de preferencia."
}

def tool_busqueda_kb(query: str) -> Dict[str, Any]:
    q = query.lower()
    hits = []
    for k, v in BASE_KB.items():
        if k in q or any(tok in v.lower() for tok in q.split()):
            hits.append({"key": k, "texto": v})
    return {"ok": True, "hits": hits[:3]}

TOOLS = {
    "calculadora": tool_calculadora,
    "busqueda_kb": tool_busqueda_kb,
}


#### **4.3 Memoria mínima**
Distinguimos dos tipos útiles:

- **memoria de trabajo**: contexto actual de la tarea
- **memoria episódica**: resultados previos (por ejemplo, cálculos, respuestas confirmadas)

Aquí usamos una implementación simple para experimentar.


In [ ]:
# Memoria simple
@dataclass
class MemoriaAgente:
    trabajo: Dict[str, Any] = field(default_factory=dict)
    episodica: List[Dict[str, Any]] = field(default_factory=list)

    def guardar_hecho(self, clave: str, valor: Any):
        self.trabajo[clave] = valor

    def registrar_evento(self, evento: Dict[str, Any]):
        self.episodica.append(evento)

    def buscar_eventos(self, keyword: str) -> List[Dict[str, Any]]:
        k = keyword.lower()
        return [e for e in self.episodica if k in str(e).lower()]

mem = MemoriaAgente()
mem.guardar_hecho("tema", "alineamiento")
mem.registrar_evento({"accion": "consulta", "contenido": "Qué es DPO"})
mem.registrar_evento({"accion": "tool", "nombre": "busqueda_kb", "query": "dpo"})
mem


#### **4.4 Planificador simple (heurístico)**
En investigación, el planificador puede ser:
- heurístico (reglas),
- prompt-based (ReAct/Plan-and-Execute),
- aprendido (policy planner).

Aquí usamos un planificador heurístico para aislar variables.


In [ ]:
# Planificador heurístico basado en palabras clave
def planificar_tarea(pregunta: str) -> List[Dict[str, Any]]:
    q = pregunta.lower()
    plan = []

    if any(sym in q for sym in ["+", "-", "*", "/"]):
        plan.append({"accion": "usar_tool", "tool": "calculadora", "arg": pregunta})

    if any(k in q for k in ["rlhf", "dpo", "orpo", "alineamiento"]):
        plan.append({"accion": "usar_tool", "tool": "busqueda_kb", "arg": pregunta})

    if not plan:
        plan.append({"accion": "respuesta_directa", "arg": pregunta})

    return plan

preguntas_demo = [
    "¿Qué es DPO y ORPO?",
    "Calcula 12 * (5 + 3)",
    "Explica instruction tuning"
]

for p in preguntas_demo:
    print(p)
    print(planificar_tarea(p))
    print()


#### **4.5 Mini-agente ejecutor**

Este agente:

1. genera un plan,
2. ejecuta herramientas,
3. guarda memoria,
4. produce respuesta final.

No usa un LLM real, solo sirve para estudiar **estructura, logs y evaluación**.


In [ ]:
def agente_toy(pregunta: str, memoria: Optional[MemoriaAgente] = None) -> Dict[str, Any]:
    memoria = memoria or MemoriaAgente()
    traza = TrazaAgente()

    traza.log(f"Pregunta recibida: {pregunta}")
    plan = planificar_tarea(pregunta)
    traza.log(f"Plan generado: {plan}")

    observaciones = []

    for paso in plan:
        if paso["accion"] == "usar_tool":
            nombre = paso["tool"]
            arg = paso["arg"]
            traza.log(f"Ejecutando herramienta: {nombre}")
            salida = TOOLS[nombre](arg)
            traza.registrar_tool(nombre, {"arg": arg}, salida)
            memoria.registrar_evento({"tool": nombre, "arg": arg, "salida": salida})
            observaciones.append((nombre, salida))

        elif paso["accion"] == "respuesta_directa":
            traza.log("No se requiere herramienta; respuesta directa.")
            observaciones.append(("directo", {"texto": "Respuesta conceptual: revisa base teórica del cuaderno."}))

    # Síntesis final 
    if observaciones and observaciones[0][0] == "calculadora":
        out = observaciones[0][1]
        if out["ok"]:
            respuesta = f"Resultado: {out['resultado']}"
        else:
            respuesta = f"No pude calcular: {out['error']}"
    elif observaciones and observaciones[0][0] == "busqueda_kb":
        hits = observaciones[0][1]["hits"]
        if hits:
            respuesta = "Resumen encontrado: " + " | ".join(h["texto"] for h in hits)
        else:
            respuesta = "No encontré información en la KB local."
    else:
        respuesta = "Respuesta directa."

    memoria.registrar_evento({"tipo": "respuesta_final", "pregunta": pregunta, "respuesta": respuesta})
    return {"respuesta": respuesta, "traza": traza, "memoria": memoria}

# Demo
mem_demo = MemoriaAgente()
res = agente_toy("¿Qué es DPO en alineamiento?", mem_demo)
print("Respuesta:", res["respuesta"])
print("\nPasos:")
for p in res["traza"].pasos:
    print("-", p)
print("\nTool calls:")
for tc in res["traza"].tool_calls:
    print(tc)


#### **4.6 Agentes con LLM real (plantilla)**
Para pasar del agente a uno real, puedes usar:

- `transformers` + parsing manual
- `LangChain`
- `LlamaIndex`
- frameworks de agentes (según tu stack)

#### **Diseño experimental recomendado**

Compara al menos:
- **LLM solo**
- **LLM + herramientas**
- **LLM + herramientas + memoria**
- **LLM + herramientas + memoria + planificación explícita**


In [ ]:
# Plantilla de protocolo para evaluar agentes (pseudo-código estructurado)
protocolo_agentes = {
    "baselines": [
        "llm_solo",
        "llm_tools",
        "llm_tools_memoria",
        "llm_tools_memoria_plan"
    ],
    "metricas": [
        "task_success_rate",
        "tool_precision",
        "tool_recall",
        "pasos_promedio",
        "latencia_total",
        "errores_de_formato",
        "hallucination_rate"
    ],
    "datasets_tarea": [
        "QA con cálculo",
        "QA con base de conocimiento",
        "extracción estructurada",
        "multi-step planning"
    ]
}
protocolo_agentes


### **5. Metodología de evaluación para investigación**


#### **5.1 Matriz de experimentos**

Una forma práctica de organizar experimentos:

- **Eje A (modelo):** base/SFT/DPO/ORPO  
- **Eje B (prompt):** corto/estructurado/con restricciones  
- **Eje C (contexto):** 512/1024/2048 tokens  
- **Eje D (agente):** sin tools / con tools / con memoria  

No cambies 5 cosas a la vez.


In [ ]:
# Plantilla de matriz experimental
import itertools
import pandas as pd

eje_modelo = ["base", "sft", "dpo", "orpo"]
eje_prompt = ["simple", "estructurado"]
eje_ctx = [512, 1024]
eje_agente = ["sin_tools", "con_tools"]

experimentos = []
for m, p, c, a in itertools.product(eje_modelo, eje_prompt, eje_ctx, eje_agente):
    exp_id = f"{m}__{p}__ctx{c}__{a}"
    experimentos.append({
        "exp_id": exp_id,
        "modelo": m,
        "prompt": p,
        "ctx": c,
        "agente": a,
        "estado": "pendiente"
    })

df_exp = pd.DataFrame(experimentos)
df_exp.head(10)


#### **5.2 Intervalos de confianza (bootstrap)**
No reportes solo un promedio. Reporta incertidumbre (aunque sea simple).


In [ ]:
# Bootstrap para una métrica (ejemplo: accuracy/win-rate)
def bootstrap_ci(valores: List[float], n_boot: int = 2000, alpha: float = 0.05, seed: int = 42):
    rng = np.random.default_rng(seed)
    arr = np.array(valores, dtype=float)
    boots = []
    for _ in range(n_boot):
        sample = rng.choice(arr, size=len(arr), replace=True)
        boots.append(sample.mean())
    lo = np.quantile(boots, alpha/2)
    hi = np.quantile(boots, 1-alpha/2)
    return float(arr.mean()), float(lo), float(hi)

valores_demo = [1,1,0,1,1,0,1,1,1,0,1,1]  # éxito/fallo por tarea
mean_, lo_, hi_ = bootstrap_ci(valores_demo)
print(f"Win-rate medio={mean_:.3f} | IC95%=({lo_:.3f}, {hi_:.3f})")


#### **5.3 Registro de errores y análisis cualitativo**

Toda investigación con LLMs debe incluir **análisis de errores**.  
Sugiere una tabla con:

- `id_ejemplo`
- `prompt`
- `prediccion`
- `gold`
- `tipo_error`
- `causa_probable`
- `propuesta_fix`


In [ ]:
# Plantilla de tabla de errores
errores = pd.DataFrame([
    {
        "id_ejemplo": 1,
        "prompt": "Devuelve JSON con 3 frutas",
        "prediccion": "manzana, pera, uva",
        "gold": '["manzana","pera","uva"]',
        "tipo_error": "formato",
        "causa_probable": "prompt poco restrictivo",
        "propuesta_fix": "forzar esquema JSON y ejemplos few-shot"
    },
    {
        "id_ejemplo": 2,
        "prompt": "Explica DPO en 2 líneas",
        "prediccion": "DPO es RLHF con PPO",
        "gold": "DPO optimiza preferencias sin PPO clásico",
        "tipo_error": "conceptual",
        "causa_probable": "confusión terminológica",
        "propuesta_fix": "agregar datos de preferencia curados + evaluación factual"
    }
])

errores


### **6. Líneas de investigación y proyectos sugeridos**


#### **6.1 Preguntas de investigación concretas**
##### **A. Tokenización y contexto**
1. ¿Qué proporción del dataset se trunca a 512/1024/2048 tokens?
2. ¿El truncamiento afecta más a ciertas clases o tareas?
3. ¿Chunking con solapamiento mejora recuperación de información?

##### **B. Instruction tuning**
4. ¿Cuál template de instrucción produce mejor cumplimiento de formato?
5. ¿Qué mezcla de datasets (general + dominio) mejora más?
6. ¿LoRA vs QLoRA cambia calidad o solo costo?

##### **C. Alineamiento**
7. ¿DPO mejora win-rate sin degradar exactitud factual?
8. ¿ORPO reduce errores de formato mejor que SFT puro?
9. ¿El modelo se vuelve demasiado "complaciente" tras alinear?

##### **D. Agentes**
10. ¿Las herramientas mejoran éxito real o solo aumentan pasos?
11. ¿Qué tipo de memoria ayuda más (episódica vs resumen)?
12. ¿La planificación explícita reduce llamadas innecesarias?


### **7. Ejercicios orientados a investigación**


#### **Ejercicios (sin código obligatorio)**

1. Define una hipótesis sobre **ventana de contexto** (por ejemplo, 512 vs 1024) y diseña un experimento con variables controladas.  
2. Diseña una **rúbrica humana** (4 criterios, escala 1-5) para evaluar respuestas de instruction tuning.  
3. Propón un esquema de dataset de **preferencias** para tu dominio (educación, salud, soporte, etc.).  
4. Explica qué casos podrían sesgar un modelo de preferencia si los anotadores no tienen guía clara.  
5. Diseña una comparación justa entre **SFT**, **DPO** y **ORPO** (mismas seeds, mismos prompts, mismo set de evaluación).  
6. Propón una arquitectura de **agente** para una tarea real (por ejemplo,análisis académico, tutor inteligente, soporte técnico).  
7. Define métricas para evaluar "buen uso de herramientas" (no solo exactitud final).  
8. Diseña una tabla de **análisis de errores** con al menos 5 tipos de fallo de LLMs.  
9. Propón un plan de mitigación para respuestas factualmente incorrectas tras el alineamiento.  
10. Redacta una sección de "amenazas a la validez" para tu experimento.


#### **Ejercicios de codificación**

1. Implementa una función que calcule **% de truncamiento** por ejemplo y por dataset, y grafique su distribución.  
2. Implementa un **formatter** de prompts (3 templates distintos) y compara longitud promedio en tokens.  
3. Implementa una evaluación automática de **JSON compliance** con esquema (campos requeridos y tipos).  
4. Implementa una versión de **DPO loss** que reciba logits reales de un modelo pequeño.  
5. Extiende el `agente_toy` para que use **memoria episódica** al responder preguntas repetidas.


In [ ]:
## Tus respuestas